In [1]:
!%pip install "pymongo[srv]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 4.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.5/485.5 kB 5.9 MB/s eta 0:00:00a 0:00:01


In [2]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

In [22]:
# Define the MongoDB server details
host = 'localhost'
port = 27017
username = 'devroot'  # Replace with your MongoDB username
password = 'devroot'  # Replace with your MongoDB password

In [23]:
# Create the connection string
connection_string = f'mongodb://{username}:{password}@{host}:{port}'

In [24]:
# Connect to the MongoDB server
client = MongoClient(connection_string)

In [25]:
try:
    # Verify connection
    client.admin.command('ping')
    print("Connected successfully to MongoDB")
    
    # List all databases
    databases = client.list_database_names()
    print("Databases:", databases)
    
    # Connect to a specific database
    db = client['your_database']  # Replace with your database name
    
    # List all collections in the database
    collections = db.list_collection_names()
    print("Collections in 'your_database':", collections)
    
    # Example: Connect to a specific collection and insert a document
    collection = db['your_collection']  # Replace with your collection name
    document = {"name": "John Doe", "age": 30, "city": "New York"}
    insert_result = collection.insert_one(document)
    print("Inserted document ID:", insert_result.inserted_id)
    
    # Find a document in the collection
    found_document = collection.find_one({"name": "John Doe"})
    print("Found document:", found_document)
    
except ConnectionFailure as e:
    print(f"Could not connect to MongoDB: {e}")
finally:
    # Close the connection
    client.close()

Connected successfully to MongoDB
Databases: ['admin', 'config', 'local']
Collections in 'your_database': []
Inserted document ID: 666c471cb1b45ccdee74db7c
Found document: {'_id': ObjectId('666c471cb1b45ccdee74db7c'), 'name': 'John Doe', 'age': 30, 'city': 'New York'}
